In [ ]:
!pip install opencv-contrib-python-headless

In [ ]:
import numpy as np
import cv2

def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 125, 255, cv2.THRESH_BINARY)
    return thresh

def detect_potential_markers(thresh):
    contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    potential_markers = []
    for contour in contours:
        potential_markers.append(contour)
    return potential_markers

def verify_and_refine_markers(potential_markers, gray_image):
    verified_markers = []
    for marker in potential_markers:
        if verify_marker(marker, gray_image):
            refined_marker = refine_corners(marker, gray_image)
            verified_markers.append(refined_marker)
    return verified_markers

def verify_marker(marker, gray_image):
    pass

def detect_aruco_markers(image):
    preprocessed = preprocess_image(image)
    potential_markers = detect_potential_markers(preprocessed)
    verified_markers = verify_and_refine_markers(potential_markers, preprocessed)
    decoded_markers = [decode_marker(marker, preprocessed) for marker in verified_markers]
    return decoded_markers

image = cv2.imread("test.jpg")
detected_markers = detect_aruco_markers(image)


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

def preprocess_image(image_path):
    image = cv2.imread(image_path)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 11, 2)

    return thresh

thresh_image = preprocess_image('test.jpg')

cv2_imshow(thresh_image)



In [ ]:
def detect_potential_markers(thresh_image):
    contours, _ = cv2.findContours(thresh_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    potential_markers = []

    for contour in contours:
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)
        if len(approx) == 4:
            area = cv2.contourArea(contour)
            if minimum_area_threshold < area < maximum_area_threshold:
                potential_markers.append(approx)

    return potential_markers

image_resolution = (1390, 857)
image_area = image_resolution[0] * image_resolution[1]

min_area_percentage = 0.01
max_area_percentage = 0.1

minimum_area_threshold = min_area_percentage * image_area
maximum_area_threshold = max_area_percentage * image_area

thresh_image = preprocess_image('test.jpg')
potential_markers = detect_potential_markers(thresh_image)

for marker in potential_markers:
    cv2.drawContours(thresh_image, [marker], -1, (255, 0, 0), 2)

cv2_imshow(thresh_image)


In [ ]:
def refine_corners(potential_markers, gray_image):
    refined_markers = []
    for marker in potential_markers:
        sorted_corners = sort_corners(marker)
        refined_markers.append(sorted_corners)
    return refined_markers

def sort_corners(corners):
    center = np.mean(corners, axis=0)
    sorted_corners = sorted(corners, key=lambda c: -np.arctan2(c[0][1] - center[0][1], c[0][0] - center[0][0]))

    top_left = min(sorted_corners, key=lambda c: c[0][0] + c[0][1])
    top_left_index = sorted_corners.index(top_left)
    sorted_corners = sorted_corners[top_left_index:] + sorted_corners[:top_left_index]

    return sorted_corners

refined_markers = refine_corners(potential_markers, thresh_image)
for marker in refined_markers:
    cv2.polylines(thresh_image, [np.array(marker)], isClosed=True, color=(255, 0, 0), thickness=2)

cv2_imshow(thresh_image)

In [ ]:
import numpy as np
import cv2

def refine_corners(potential_markers, gray_image):
    refined_markers = []
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)

    for contour in potential_markers:
        corners = np.float32(contour).reshape(-1, 2)
        cv2.cornerSubPix(gray_image, corners, (5, 5), (-1, -1), criteria)

        refined_markers.append(corners)

    return refined_markers

gray_image = np.float32(thresh_image)

refined_markers = refine_corners(potential_markers, gray_image)

for marker in refined_markers:
    marker = marker.reshape(-1, 1, 2)
    cv2.polylines(thresh_image, [np.int32(marker)], True, (255, 0, 0), 2)

cv2_imshow(thresh_image)

In [ ]:
def decode_marker(image, refined_markers):
    decoded_markers = []
    marker_side_length = 6
    for corners in refined_markers:
        source_points = np.array(corners, dtype='float32')
        destination_points = np.array([[0, 0], [marker_side_length - 1, 0],
                                       [marker_side_length - 1, marker_side_length - 1], [0, marker_side_length - 1]],
                                       dtype='float32')

        transform_matrix = cv2.getPerspectiveTransform(source_points, destination_points)
        marker_image = cv2.warpPerspective(image, transform_matrix, (marker_side_length, marker_side_length))

        _, binary_marker_image = cv2.threshold(marker_image, 125, 255, cv2.THRESH_BINARY)

        id = decode_id_from_binary_image(binary_marker_image[1:-1, 1:-1])  # exclude the border

        if id is not None:
            decoded_markers.append((id, corners))

    return decoded_markers

def decode_id_from_binary_image(binary_image):
    grid_size = 4
    step = binary_image.shape[0] // grid_size
    id_bits = []

    for y in range(0, binary_image.shape[0], step):
        for x in range(0, binary_image.shape[1], step):
            cell = binary_image[y:y+step, x:x+step]
            bit = int(np.mean(cell) > 127)
            id_bits.append(bit)

    id = bits_to_id(id_bits)
    return id

def bits_to_id(bits):
    #DICT_5X5_250
    pass

gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
decoded_markers = decode_marker(gray_image, refined_markers)

for id, corners in decoded_markers:
    print(f"Decoded marker ID: {id}")
    corners = np.int0(corners)
    cv2.polylines(image, [corners], True, (0, 255, 0), 2)

cv2_imshow("Decoded Markers", image)


In [ ]:
import cv2
import numpy as np

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blurred, 125, 255, cv2.THRESH_BINARY_INV)
    return thresh, gray

def detect_potential_markers(thresh_image):
    contours, _ = cv2.findContours(thresh_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    potential_markers = []
    for contour in contours:
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)
        if len(approx) == 4:
            area = cv2.contourArea(contour)
            if minimum_area_threshold < area < maximum_area_threshold:
                potential_markers.append(approx)
    return potential_markers

def refine_corners(potential_markers, gray_image):
    refined_markers = []
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    for contour in potential_markers:
        corners = np.float32(contour).reshape(-1, 2)
        cv2.cornerSubPix(gray_image, corners, (5, 5), (-1, -1), criteria)
        refined_markers.append(corners)
    return refined_markers

def decode_marker(gray_image, refined_markers):
    decoded_markers = []
    marker_side_length = 50

    for corners in refined_markers:
        source_points = corners.reshape(4, 2)
        destination_points = np.array([[0, 0], [marker_side_length - 1, 0],
                                       [marker_side_length - 1, marker_side_length - 1], [0, marker_side_length - 1]],
                                       dtype="float32")

        transform_matrix = cv2.getPerspectiveTransform(source_points, destination_points)
        marker_image = cv2.warpPerspective(gray_image, transform_matrix, (marker_side_length, marker_side_length))

        _, binary_marker_image = cv2.threshold(marker_image, 125, 255, cv2.THRESH_BINARY)

        id_bits = extract_id_from_image(binary_marker_image[4:-4, 4:-4], grid_size=4)
        marker_id = bits_to_id(id_bits)
        if marker_id is not None:
            decoded_markers.append((marker_id, corners))

    return decoded_markers

def extract_id_from_image(marker_image, grid_size):
    cell_size = marker_image.shape[0] // grid_size
    id_bits = []

    for y in range(0, marker_image.shape[0], cell_size):
        for x in range(0, marker_image.shape[1], cell_size):
            cell = marker_image[y:y+cell_size, x:x+cell_size]
            id_bit = int(np.mean(cell) > 127)
            id_bits.append(id_bit)

    return id_bits

def bits_to_id(bits):
    id = 0
    for bit in bits:
        id = (id << 1) | bit
    return id

min_area_percentage = 0.01
max_area_percentage = 0.1

# Example usage
image_path = 'test.jpg'
thresh_image, gray_image = preprocess_image(image_path)
image_area = thresh_image.shape[0] * thresh_image.shape[1]
minimum_area_threshold = min_area_percentage * image_area
maximum_area_threshold = max_area_percentage * image_area

potential_markers = detect_potential_markers(thresh_image)
refined_markers = refine_corners(potential_markers, gray_image)
decoded_markers = decode_marker(gray_image, refined_markers)

for marker_id, corners in decoded_markers:
    print(f"Decoded marker ID: {marker_id}")
    corners = np.int0(corners)
    cv2.polylines(thresh_image, [corners], True, (0, 255, 0), 2)

cv2_imshow(thresh_image)

for marker_id, corners in decoded_markers:
    print(f"Decoded marker ID: {marker_id}")

    int_corners = np.int0(corners)

    cv2.polylines(thresh_image, [int_corners], True, (0, 255, 0), 2)

cv2_imshow(thresh_image)


In [33]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

image_path = 'singlemarkersoriginal.jpg'
image = cv2.imread(image_path)
image_area = image.shape[0] * image.shape[1]

marker_width_estimate = image.shape[1] / 20
marker_area_estimate = marker_width_estimate**2

min_area_factor = 0.5
max_area_factor = 1.5
minimum_area_threshold = min_area_factor * marker_area_estimate
maximum_area_threshold = max_area_factor * marker_area_estimate

print(f"Estimated marker area: {marker_area_estimate}")
print(f"Minimum area threshold: {minimum_area_threshold}")
print(f"Maximum area threshold: {maximum_area_threshold}")


Estimated marker area: 1024.0
Minimum area threshold: 512.0
Maximum area threshold: 1536.0


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blurred, 125, 255, cv2.THRESH_BINARY_INV)
    return thresh, gray

def detect_potential_markers(thresh_image):
    contours, _ = cv2.findContours(thresh_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    potential_markers = []
    for contour in contours:
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)
        if len(approx) == 4:
            area = cv2.contourArea(contour)
            if minimum_area_threshold < area < maximum_area_threshold:
                potential_markers.append(approx)
    return potential_markers

def refine_corners(potential_markers, gray_image):
    refined_markers = []
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    for contour in potential_markers:
        corners = np.float32(contour).reshape(-1, 2)
        cv2.cornerSubPix(gray_image, corners, (5, 5), (-1, -1), criteria)
        refined_markers.append(corners)
    return refined_markers

image_path = 'test.jpg'
thresh_image, gray_image = preprocess_image(image_path)
image_area = thresh_image.shape[0] * thresh_image.shape[1]

minimum_area_threshold = image_area / 2 * 0.8
maximum_area_threshold = image_area / 2 * 1.2

print(f"Minimum area threshold: {minimum_area_threshold}")
print(f"Maximum area threshold: {maximum_area_threshold}")

potential_markers = detect_potential_markers(thresh_image)
print(f"Detected {len(potential_markers)} potential markers.")

refined_markers = refine_corners(potential_markers, gray_image)
print(f"Refined {len(refined_markers)} potential markers.")

for corners in refined_markers:
    int_corners = np.int0(corners)
    cv2.polylines(thresh_image, [int_corners], True, (0, 255, 0), 2)

cv2_imshow(thresh_image)


In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    _, thresh = cv2.threshold(blurred, 125, 255, cv2.THRESH_BINARY_INV)
    return thresh, gray

def detect_potential_markers(thresh_image):
    contours, _ = cv2.findContours(thresh_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    potential_markers = []
    for contour in contours:
        perimeter = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)
        if len(approx) == 4:
            area = cv2.contourArea(contour)
            if minimum_area_threshold < area < maximum_area_threshold:
                potential_markers.append(approx)
    return potential_markers

def refine_corners(potential_markers, gray_image):
    refined_markers = []
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    for contour in potential_markers:
        corners = np.float32(contour).reshape(-1, 2)
        cv2.cornerSubPix(gray_image, corners, (5, 5), (-1, -1), criteria)
        refined_markers.append(corners)
    return refined_markers

image_path = 'test.jpg'
thresh_image, gray_image = preprocess_image(image_path)

image_area = gray_image.shape[0] * gray_image.shape[1]
minimum_area_threshold = image_area * 0.25
maximum_area_threshold = image_area * 0.75

print(f"Minimum area threshold: {minimum_area_threshold}")
print(f"Maximum area threshold: {maximum_area_threshold}")

potential_markers = detect_potential_markers(thresh_image)
print(f"Detected {len(potential_markers)} potential markers.")

refined_markers = refine_corners(potential_markers, gray_image)
print(f"Refined {len(refined_markers)} potential markers.")

for corners in refined_markers:
    int_corners = np.int0(corners)
    cv2.polylines(thresh_image, [int_corners], True, (0, 255, 0), 2)

cv2_imshow(thresh_image)
